In [1]:
# Import dependencies
import gmaps
import requests
import pandas as pd

# import the API key
from config import g_key


In [2]:
# Store the CSV you saved created in part one into a DataFrame
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Busselton,-33.6500,115.3333,57.27,77,98,22.41,AU,2022-08-07 22:20:03
1,1,Yellowknife,62.4560,-114.3525,63.12,67,75,4.61,CA,2022-08-07 22:17:01
2,2,Upernavik,72.7868,-56.1549,42.46,83,84,8.30,GL,2022-08-07 22:20:04
3,3,Saint Anthony,45.0205,-93.2180,75.25,73,100,9.22,US,2022-08-07 22:15:29
4,4,Najran,17.4924,44.1277,83.05,44,9,7.18,SA,2022-08-07 22:20:04


In [3]:
# Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [4]:
# configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])

In [7]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 85


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Busselton,-33.6500,115.3333,57.27,77,98,22.41,AU,2022-08-07 22:20:03
1,1,Yellowknife,62.4560,-114.3525,63.12,67,75,4.61,CA,2022-08-07 22:17:01
3,3,Saint Anthony,45.0205,-93.2180,75.25,73,100,9.22,US,2022-08-07 22:15:29
4,4,Najran,17.4924,44.1277,83.05,44,9,7.18,SA,2022-08-07 22:20:04
5,5,Rikitea,-23.1203,-134.9692,72.52,69,100,32.12,PF,2022-08-07 22:20:04
6,6,Guilin,25.2819,110.2864,76.23,90,87,0.76,CN,2022-08-07 22:20:05
8,8,Rivers,50.0308,-100.2403,72.55,39,0,7.25,CA,2022-08-07 22:20:05
9,9,Carnarvon,-24.8667,113.6333,61.75,79,0,11.25,AU,2022-08-07 22:20:06
12,12,Maceio,-9.6658,-35.7353,74.64,83,20,5.75,BR,2022-08-07 22:20:06
17,17,Nantucket,41.2835,-70.0995,82.00,85,0,16.11,US,2022-08-07 22:20:08


In [11]:
preferred_cities_df.count()

City_ID       408
City          408
Lat           408
Lng           408
Max Temp      408
Humidity      408
Cloudiness    408
Wind Speed    408
Country       405
Date          408
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Busselton,AU,57.27,-33.6500,115.3333,
1,Yellowknife,CA,63.12,62.4560,-114.3525,
3,Saint Anthony,US,75.25,45.0205,-93.2180,
4,Najran,SA,83.05,17.4924,44.1277,
5,Rikitea,PF,72.52,-23.1203,-134.9692,
6,Guilin,CN,76.23,25.2819,110.2864,
8,Rivers,CA,72.55,50.0308,-100.2403,
9,Carnarvon,AU,61.75,-24.8667,113.6333,
12,Maceio,BR,74.64,-9.6658,-35.7353,
17,Nantucket,US,82.00,41.2835,-70.0995,


In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [14]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel Not Found... skipping")
        print(hotel_df)

Hotel Not Found... skipping
              City Country  Max Temp      Lat       Lng  \
0        Busselton      AU     57.27 -33.6500  115.3333   
1      Yellowknife      CA     63.12  62.4560 -114.3525   
3    Saint Anthony      US     75.25  45.0205  -93.2180   
4           Najran      SA     83.05  17.4924   44.1277   
5          Rikitea      PF     72.52 -23.1203 -134.9692   
..             ...     ...       ...      ...       ...   
588        Batagay      RU     67.66  67.6333  134.6333   
589    Yenangyaung      MM     77.27  20.4667   94.8833   
591         Talara      PE     64.13  -4.5772  -81.2719   
592     Shitanjing      CN     73.13  39.2342  106.3439   
593        Bontang      ID     75.58   0.1333  117.5000   

                                 Hotel Name  
0                   Observatory Guest House  
1                        The Explorer Hotel  
3    Holiday Inn Express Roseville-St. Paul  
4                         محمد هادي ال فطيح  
5                           Peopl

Hotel Not Found... skipping
              City Country  Max Temp      Lat       Lng  \
0        Busselton      AU     57.27 -33.6500  115.3333   
1      Yellowknife      CA     63.12  62.4560 -114.3525   
3    Saint Anthony      US     75.25  45.0205  -93.2180   
4           Najran      SA     83.05  17.4924   44.1277   
5          Rikitea      PF     72.52 -23.1203 -134.9692   
..             ...     ...       ...      ...       ...   
588        Batagay      RU     67.66  67.6333  134.6333   
589    Yenangyaung      MM     77.27  20.4667   94.8833   
591         Talara      PE     64.13  -4.5772  -81.2719   
592     Shitanjing      CN     73.13  39.2342  106.3439   
593        Bontang      ID     75.58   0.1333  117.5000   

                                 Hotel Name  
0                   Observatory Guest House  
1                        The Explorer Hotel  
3    Holiday Inn Express Roseville-St. Paul  
4                         محمد هادي ال فطيح  
5                           Peopl

Hotel Not Found... skipping
              City Country  Max Temp      Lat       Lng  \
0        Busselton      AU     57.27 -33.6500  115.3333   
1      Yellowknife      CA     63.12  62.4560 -114.3525   
3    Saint Anthony      US     75.25  45.0205  -93.2180   
4           Najran      SA     83.05  17.4924   44.1277   
5          Rikitea      PF     72.52 -23.1203 -134.9692   
..             ...     ...       ...      ...       ...   
588        Batagay      RU     67.66  67.6333  134.6333   
589    Yenangyaung      MM     77.27  20.4667   94.8833   
591         Talara      PE     64.13  -4.5772  -81.2719   
592     Shitanjing      CN     73.13  39.2342  106.3439   
593        Bontang      ID     75.58   0.1333  117.5000   

                                 Hotel Name  
0                   Observatory Guest House  
1                        The Explorer Hotel  
3    Holiday Inn Express Roseville-St. Paul  
4                         محمد هادي ال فطيح  
5                           Peopl

Hotel Not Found... skipping
              City Country  Max Temp      Lat       Lng  \
0        Busselton      AU     57.27 -33.6500  115.3333   
1      Yellowknife      CA     63.12  62.4560 -114.3525   
3    Saint Anthony      US     75.25  45.0205  -93.2180   
4           Najran      SA     83.05  17.4924   44.1277   
5          Rikitea      PF     72.52 -23.1203 -134.9692   
..             ...     ...       ...      ...       ...   
588        Batagay      RU     67.66  67.6333  134.6333   
589    Yenangyaung      MM     77.27  20.4667   94.8833   
591         Talara      PE     64.13  -4.5772  -81.2719   
592     Shitanjing      CN     73.13  39.2342  106.3439   
593        Bontang      ID     75.58   0.1333  117.5000   

                                 Hotel Name  
0                   Observatory Guest House  
1                        The Explorer Hotel  
3    Holiday Inn Express Roseville-St. Paul  
4                         محمد هادي ال فطيح  
5                           Peopl

Hotel Not Found... skipping
              City Country  Max Temp      Lat       Lng  \
0        Busselton      AU     57.27 -33.6500  115.3333   
1      Yellowknife      CA     63.12  62.4560 -114.3525   
3    Saint Anthony      US     75.25  45.0205  -93.2180   
4           Najran      SA     83.05  17.4924   44.1277   
5          Rikitea      PF     72.52 -23.1203 -134.9692   
..             ...     ...       ...      ...       ...   
588        Batagay      RU     67.66  67.6333  134.6333   
589    Yenangyaung      MM     77.27  20.4667   94.8833   
591         Talara      PE     64.13  -4.5772  -81.2719   
592     Shitanjing      CN     73.13  39.2342  106.3439   
593        Bontang      ID     75.58   0.1333  117.5000   

                                 Hotel Name  
0                   Observatory Guest House  
1                        The Explorer Hotel  
3    Holiday Inn Express Roseville-St. Paul  
4                         محمد هادي ال فطيح  
5                           Peopl

In [15]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [16]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [17]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))